In [ ]:
!pip install haversine
!pip install simanneal
!pip install pyswarms
!pip install pymoo
!pip install folium
!pip install geoplot
!pip install matplotlib
import time

import numpy as np
import pandas as pd
from collections import defaultdict
from haversine import haversine
import networkx as nx
import matplotlib.pyplot as plt
import random
import math
import time
import pyswarms as ps
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import geoplot
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap


In [ ]:
#here we select some sample location points for better visulize how the algorithm works
city_dic = {
    'Thorncliffe Parl Public School': (43.70548545348342, -79.3430794474666),
    'Student 1 Roanoke Apt': (43.74105199944502, -79.32850826280696),

    'student 3 Lawrance Apt': (43.738524163584984, -79.34149211798413),
    'student 4 Super Sonic Condos':(43.71957929031141, -79.33595881597378),
    'Intersection 1 (Eglinton&Don Mills)':(43.7207386416841, -79.33882906166014),
    'Intersection 2 (Lawrence&Don Mills)':(43.737422024371426, -79.34352888079776),
    'DVP_Enter/End':(43.72331679647265, -79.33020300398714),
    'DVP_End/Enter':(43.737037, -79.331921)
}
    # 'Intersection 3 (Beth & Wicksteed)':(43.713378, -79.346490),
    # 'Intersection 4 (Wicksteed & Brentcliffe)':(43.712899, -79.350669),
    # 'Inter 5 (Brentcliffe & Eglinton)':(43.714640, -79.359839),
    # 'student 2 H&R Apt': (43.727444179004465, -79.32466485355351),


# center the map around the first location in the dictionary
first_location_name, first_location_coords = next(iter(city_dic.items()))

# Create a map object centered around the first location
m = folium.Map(location=first_location_coords, zoom_start=13)

# Iterate over each item in the city_dic and add markers on the map
#The red marker represents the starting/ending point: Thorncliffe Park Public School
for place, coords in city_dic.items():
    # Check if the current place is "Thorncliffe Park Public School"
    if place == 'Thorncliffe Park Public School':
        # If it is, create a marker with a red icon
        folium.Marker(
            location=coords,
            popup=place,
            icon=folium.Icon(color='red')  # Red color for the school
        ).add_to(m)
    else:
        # For all other places, use blue icons
        folium.Marker(
            location=coords,
            popup=place,
            icon=folium.Icon(color='blue')
        ).add_to(m)

# Display the map
m

In [ ]:
# weather config
current_weather = 1 # 0 means good weather, 1 means snow or rainy weather

In [ ]:


#Define the function for heavy-traffic and high-crime roads with a relatively high weight.
#Note here city means each location point
#We assign different weight on traffic_level, criminal_level, weather

def get_traffic_level(city1, city2):
    if (city1, city2) in [('DVP_Enter/End', 'DVP_End/Enter'), ('DVP_End/Enter', 'DVP_Enter/End')]:
        # return float('inf')
        return 10  # Here 10 represents a high traffic-level
    return 1  # 1 represents a normal traffic level for other paths

def get_criminal_level(city1, city2):
    if (city1, city2) in [('Thorncliffe Parl Public School', 'Intersection 2 (Lawrence&Don Mills)'), ('Intersection 2 (Lawrence&Don Mills)', 'Thorncliffe Parl Public School')]:
        # return float('inf')
        return 10  # Here 10 represents a high criminal-level
    return 1  # 1 represents a normal criminal level for other paths
def get_weather_level(weather):
    if (weather==1):
        return 10
    return 1

city_dic = {
    'Thorncliffe Parl Public School': (43.70548545348342, -79.3430794474666),
    'Student 1 Roanoke Apt': (43.74105199944502, -79.32850826280696),
    'student 3 Lawrance Apt': (43.738524163584984, -79.34149211798413),
    'student 4 Super Sonic Condos':(43.71957929031141, -79.33595881597378),
    'Intersection 1 (Eglinton&Don Mills)':(43.7207386416841, -79.33882906166014),
    'Intersection 2 (Lawrence&Don Mills)':(43.737422024371426, -79.34352888079776),
    'DVP_Enter/End':(43.72331679647265, -79.33020300398714),
    'DVP_End/Enter':(43.737037, -79.331921)
}
edge_labels = {
    ('DVP_End/Enter', 'DVP_Enter/End'): 'Heavy Traffic',  # Label for heavy traffic
    ('Thorncliffe Parl Public School', 'Intersection 2 (Lawrence&Don Mills)'): 'High Crime'  # Label for high crime
}
#loop the dictionary twice and check to make sure each location are not the same to avoid it connects itself
distance_2cities = defaultdict(dict)
for city1,city1_coords in city_dic.items():
  for city2, city2_coords in city_dic.items():
    if city1 == city2:
      distance_2cities[city1][city2] = 0.0
    else:
      distance_2cities[city1][city2] = haversine(city1_coords, city2_coords)

# Convert distance diccionary into a dataframe
distances_pandaFrame = pd.DataFrame(distance_2cities)
city_names=list(distances_pandaFrame.columns)#convert each column into list type. (each item in list can be anything)
distances=distances_pandaFrame.values #extract the values

Creat the map on the network graph to better visulize our pircked points and doing the analysis

In [ ]:
G=nx.Graph()#create a networking graph with nodes as cities, distance as edges
#add edges to G


# Initialize an empty dictionary for the reversed value because Network assumes data in (horizontal,vertical)
reversed_dict = {}

# Loop and reverse each value
for key, value in city_dic.items():
    reversed_dict[key] = value[::-1]

for city1, city1_coords in city_dic.items():
    for city2, city2_coords in city_dic.items():
        if city1 != city2:  # Check to ensure cities are not the same to avoid it connects itself
            distance = distance_2cities[city1][city2]
            G.add_edge(city1, city2, weight=distance)

fig, ax = plt.subplots(figsize=(15,10))

#remobve some unnecessary connections
G.remove_edge('Thorncliffe Parl Public School', 'student 3 Lawrance Apt')
G.remove_edge('Thorncliffe Parl Public School', 'Student 1 Roanoke Apt')
G.remove_edge('Thorncliffe Parl Public School', 'DVP_End/Enter')
# G.remove_edge('Intersection 2 (Lawrence&Don Mills)', 'student 2 H&R Apt')
G.remove_edge('Intersection 2 (Lawrence&Don Mills)', 'Student 1 Roanoke Apt')
G.remove_edge('student 3 Lawrance Apt', 'DVP_End/Enter')

# Create an independent copy
H = G.copy()

# Draw the network
ax=nx.draw_networkx(
    H,
    pos=reversed_dict,#pos is the coordates of nodes
    with_labels=True,#display names
    edge_color="grey",
    node_size=200,
    width=1,#line thickness of edge
)

# Draw edge labels
nx.draw_networkx_edge_labels(
    H,
    pos=reversed_dict,
    edge_labels=edge_labels,
    font_color='red',
)

plt.show()

Define Cost Functions

In [ ]:

 #define the distance and traffic_volume as an extra heuristic infromation to be considered
traffic_volume = {
    ('DVP_Enter/End', 'DVP_End/Enter'): 10,  # High traffic path with a high weight

}
criminal_info = {
    ('Thorncliffe Parl Public School',  'Intersection 2 (Lawrence&Don Mills)'):10,
}
weather_info = {
    ('Thorncliffe Parl Public School',  'student 4 Super Sonic Condos'):3,
    ('Thorncliffe Parl Public School',  'Intersection 1 (Eglinton&Don Mills)'):5,
    ('Intersection 2 (Lawrence&Don Mills)', 'student 3 Lawrance Apt'):7,
    ('Intersection 1 (Eglinton&Don Mills)', 'DVP_End/Enter'):6,
    ('DVP_End/Enter', 'Student 1 Roanoke Apt'):4,
    ('Intersection 2 (Lawrence&Don Mills)', 'Intersection 1 (Eglinton&Don Mills)'):2,
}
def get_traffic_weight(path_segment, traffic_volume):
    # Retrieve the traffic weight for a given path segment from traffic_volume
    return traffic_volume.get(path_segment, 1)

# def cost_function(path, distances, traffic_volume):
#     path_cost = 0
#     for i in range(len(path) - 1):
#         path_segment = (city_names[path[i]], city_names[path[i+1]])
#         distance = distances[path[i], path[i+1]]
#         traffic_weight = get_traffic_weight(path_segment, traffic_volume)
#         # criminal_weight = get_criminal_weight(path_segment, criminal_rate)
#         path_cost += distance * traffic_weight#*criminal_weight#update the cost with the weight of traffic volume
#     return path_cost

def get_criminal_weight(path_segment, criminal_info):
    # Retrieve the criminal weight for a given path segment from criminal_info
    return criminal_info.get(path_segment, 1)  # default criminal weight is 1

def get_weather_weight(path_segment, weather_info):
   # Retrieve the weather weight for a given path segment from weather_info
    return weather_info.get(path_segment, 1) # default weather weight is 1

#define cost function
def cost_function(path, distances, traffic_info, criminal_info, weather_info):
    path_cost = 0
    for i in range(len(path) - 1):
        path_segment = (city_names[path[i]], city_names[path[i+1]])
        distance = distances[path[i], path[i+1]]
        traffic_weight = get_traffic_weight(path_segment, traffic_info)
        criminal_weight = get_criminal_weight(path_segment, criminal_info)
        weather_weight = get_weather_weight(path_segment, weather_info)
        path_cost += distance * traffic_weight * criminal_weight * weather_weight
    return path_cost


In [ ]:
#paths is initialized as a NumPy array to store the path for each ant.
#The dimensions are (ants, city_count), representing the number of ants and the number of cities, respectively.
city_count = 8
ants = 30
alpha = 0.5
beta = 1.5 #here we assign a relatively high beta value for increaseing the power of the heuristic information
pheromones = np.ones((city_count, city_count))


def ant_tour(pheromones):
    all_possible_paths = np.empty((30, city_count+1), dtype=int) #an empty 2D array to store all possible paths. 10 is because ant go back to school (9+1)
    #for each ant:
    for ant in range(ants):
      path = [0]#all ants need to start from 1st city, let's assume it is 0 stored in a list
      unvisited_cities = list(range(1, city_count)) #unvisited_cities = [1,2,...,9]

      while unvisited_cities:
          current_city = path[-1] #the most recent city that ant visited (for itration 1, path[-1] is 0)
          probabilities = []

          for city in unvisited_cities:
            tau = pheromones[current_city, city]#here pheromones is a ones matrix

            distance_heuristic = distances[current_city, city]

            traffic_heuristic = get_traffic_level(current_city, city)
            #eta = (1/distance_heuristic)/traffic_heuristic #compute eta by including heuristic information in the denominator.
            # eta = 1/(distance_heuristic*traffic_heuristic)

            criminal_heuristic = get_criminal_level(current_city, city)

            weather_heuristic = get_weather_level(current_weather)

            eta = 1/(distance_heuristic*traffic_heuristic*criminal_heuristic*weather_heuristic)

            probabilities.append((tau** alpha)*(eta ** beta)) #compute probability of the ant to select the next path

          probabilities /= sum(probabilities)
          next_city = random.choices(unvisited_cities, weights=probabilities)[0]
          unvisited_cities.remove(next_city)
          path.append(next_city)#record the path(route)for each ant
      path.append(0)#make the ant go back to School
      all_possible_paths[ant] = path#store the recorded path into paths matrix

    return all_possible_paths

In [ ]:
evaporation_rate = 0.5
Q=100

def update_pheromones(distances_paths, all_possible_paths, pheromones):
    delta_pheromones = np.zeros((city_count, city_count))

    for i in range(ants):
        for j in range(city_count-1):
            #here update pheronmone between each two city (path)based on each ant's route
            delta_pheromones[all_possible_paths[i, j], all_possible_paths[i, j+1]] += Q / distances_paths[i]
        delta_pheromones[all_possible_paths[i, -1], all_possible_paths[i, 0]] += Q / distances_paths[i]

    update_phero_values=(1 - evaporation_rate) * pheromones + delta_pheromones
    return update_phero_values

In [ ]:
def run_ACO(distances, ants, iterations, alpha, beta, evaporation_rate, Q):
    # start_time = time.time()  # Start timing
    pheromones = np.ones((city_count, city_count))
    best_path = None
    infinity = float('inf')#set to infinity to help to find the minimum value

    for _ in range(iterations):
        # Generate paths for each ant
        all_possible_paths =ant_tour(pheromones)


        #uncomment this if you do not want the path to consider extral heuristic information of avoiding heavy traffic path
        # distances_paths = np.array([cost_function(path, distances) for path in all_possible_paths])
        distances_paths = np.array([cost_function(path, distances, traffic_volume, criminal_info, weather_info) for path in all_possible_paths])
        min_idx = distances_paths.argmin()
        min_distance = distances_paths[min_idx]

        if min_distance < infinity:
            infinity = min_distance
            best_path = all_possible_paths[min_idx]

        # Update pheromones
        update_pheromones(distances_paths, all_possible_paths, pheromones)
    # end_time = time.time()  # End timing
    # duration = end_time - start_time  # Calculate duration
    # print(all_possible_paths)
    # print("Time taken to generate the route: {:.2f} seconds".format(duration))
    return best_path, infinity


In [ ]:
total_time_list = []
cost_list = []
#write values of parameters into run_ACO function
start_time = time.time()
best_path, infinity = run_ACO(distances, ants=30, iterations=100, alpha=0.5, beta=1.5, evaporation_rate=0.5, Q=100)
total_time = time.time() - start_time
total_time_list.append(total_time)
cost_list.append(infinity)

In [ ]:
#print the route and its length
ants_tour=[]
for i in best_path:
  ants_tour.append(city_names[i])
# ants_tour=[city_names[i] for i in best_path]
route_names = " → ".join(ants_tour)
print("route_names:", route_names)
# print("route length:", np.round(infinity, 3))#round it to 3 decimal places

The figure below show the route successfully go around (detour) the high-crime road and high-traffic road to improve the safety of students

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
# H = G.copy()


# Initialize an empty dictionary for the reversed value because Network assumes data in (horizontal,vertical)
reversed_dict = {}
for key, value in city_dic.items():# Loop and reverse each value
    reversed_dict[key] = value[::-1]
#store ants_tour
edge_list =list(nx.utils.pairwise(ants_tour))
nx.draw_networkx_edges(H, pos=reversed_dict, edge_color="gray", width=0.5)
edge_labels = {
    ('DVP_End/Enter', 'DVP_Enter/End'): 'Heavy Traffic',  # Label for heavy traffic
    ('Thorncliffe Parl Public School', 'Intersection 2 (Lawrence&Don Mills)'): 'High Crime'  # Label for high crime
}
# Draw the network
ax=nx.draw_networkx(
    H,
    pos=reversed_dict,#pos is the coordates of nodes
    with_labels=True,#display names
    edgelist=edge_list, #only draw the tour that is stored in the edge_list
    edge_color="red",
    node_size=200,
    width=2,#thickness of edge
)

nx.draw_networkx_edge_labels(
    H,
    pos=reversed_dict,
    edge_labels=edge_labels,
    font_color='red',
)
plt.show()

In [ ]:
# Use the default parameter, beta=0.5
start_time = time.time()
best_path, infinity = run_ACO(distances, ants=30, iterations=100, alpha=0.5, beta=0.5, evaporation_rate=0.5, Q=100)
total_time = time.time() - start_time
total_time_list.append(total_time)
cost_list.append(infinity)

In [ ]:
#print the route and its length
ants_tour=[]
for i in best_path:
  ants_tour.append(city_names[i])
# ants_tour=[city_names[i] for i in best_path]
route_names = " → ".join(ants_tour)
print("route_names:", route_names)
# print("route length:", np.round(infinity, 3))#round it to 3 decimal places

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
# H = G.copy()


# Initialize an empty dictionary for the reversed value because Network assumes data in (horizontal,vertical)
reversed_dict = {}
for key, value in city_dic.items():# Loop and reverse each value
    reversed_dict[key] = value[::-1]
#store ants_tour
edge_list =list(nx.utils.pairwise(ants_tour))
nx.draw_networkx_edges(H, pos=reversed_dict, edge_color="gray", width=0.5)
edge_labels = {
    ('DVP_End/Enter', 'DVP_Enter/End'): 'Heavy Traffic',  # Label for heavy traffic
    ('Thorncliffe Parl Public School', 'Intersection 2 (Lawrence&Don Mills)'): 'High Crime'  # Label for high crime
}
# Draw the network
ax=nx.draw_networkx(
    H,
    pos=reversed_dict,#pos is the coordates of nodes
    with_labels=True,#display names
    edgelist=edge_list, #only draw the tour that is stored in the edge_list
    edge_color="red",
    node_size=200,
    width=2,#thickness of edge
)

nx.draw_networkx_edge_labels(
    H,
    pos=reversed_dict,
    edge_labels=edge_labels,
    font_color='red',
)
plt.show()

In [ ]:
cost_result = pd.DataFrame({'Expriment':['Default', 'EX1'], 'Cost':[
    cost_list[1],
    cost_list[0]
]})
cost_result.plot.bar(x='Expriment', y='Cost',rot=0)

In [ ]:
time_result = pd.DataFrame({'Expriment':['Default', 'EX1'], 'Time':[
    total_time_list[1],
    total_time_list[0]
]})
time_result.plot.bar(x='Expriment', y='Time', rot=0)